In [1]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.utils import np_utils
from keras import backend as K
import pandas as pd
import numpy as np
from keras.optimizers import SGD

Using TensorFlow backend.


In [2]:
data_x=pd.read_pickle('../input/mydataset/trainX.pickle')
data_y=pd.read_pickle('../input/mydataset/trainY.pickle')
trainX=data_x[:40000]
trainY=data_y[:40000]
testX=data_x[40000:]
testY=data_y[40000:]

trainX = np.expand_dims(trainX, -1)
testX = np.expand_dims(testX, -1)

In [3]:
class MiniVGGNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
        model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(32, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
 
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
 
        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
 
        model.add(Dense(classes))
        model.add(Activation("softmax"))
 
        return model

In [4]:
NUM_EPOCHS = 35
INIT_LR = 1e-2
BS = 32
if K.image_data_format() == "channels_first":
    trainX = trainX.reshape((trainX.shape[0], 1, 28, 28))
    testX = testX.reshape((testX.shape[0], 1, 28, 28))
 
else:
    trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
    testX = testX.reshape((testX.shape[0], 28, 28, 1))

In [5]:
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0
 
trainY = np_utils.to_categorical(trainY, 10)
testY = np_utils.to_categorical(testY, 10)


In [6]:
opt = SGD(lr=INIT_LR, momentum=0.9, decay=INIT_LR / NUM_EPOCHS)
model = MiniVGGNet.build(width=28, height=28, depth=1, classes=10)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
 
H = model.fit(trainX, trainY,validation_data=(testX, testY),batch_size=BS, epochs=NUM_EPOCHS)

Train on 40000 samples, validate on 20000 samples
Epoch 1/35
40000/40000 [==============================] - 18s 450us/step - loss: 0.5986 - acc: 0.8055 - val_loss: 0.3871 - val_acc: 0.8585
Epoch 2/35
40000/40000 [==============================] - 14s 338us/step - loss: 0.3853 - acc: 0.8643 - val_loss: 0.2855 - val_acc: 0.8990
Epoch 3/35
40000/40000 [==============================] - 13s 337us/step - loss: 0.3291 - acc: 0.8836 - val_loss: 0.2534 - val_acc: 0.9103
Epoch 4/35
40000/40000 [==============================] - 13s 336us/step - loss: 0.2924 - acc: 0.8954 - val_loss: 0.2381 - val_acc: 0.9126
Epoch 5/35
40000/40000 [==============================] - 14s 338us/step - loss: 0.2712 - acc: 0.9016 - val_loss: 0.2564 - val_acc: 0.9094
Epoch 6/35
40000/40000 [==============================] - 13s 337us/step - loss: 0.2592 - acc: 0.9061 - val_loss: 0.2197 - val_acc: 0.9209
Epoch 7/35
40000/40000 [==============================] - 14s 340us/step - loss: 0.2483 - acc: 0.9105 - val_loss: 0.

In [7]:
preds = model.predict(testX)

In [8]:
ans=pd.DataFrame(preds)

In [9]:
ans.head()

,0,1,2,3,4,5,6,7,8,9
0,1.718901e-09,2.990237e-08,3.580937e-10,3.985034e-08,1.400814e-09,6.706579e-06,2.799786e-09,9.997165e-01,1.697622e-08,2.767327e-04
1,2.452660e-08,1.011674e-08,1.484065e-08,9.424107e-08,2.534328e-07,1.303684e-04,6.056143e-08,9.995827e-01,6.520906e-08,2.864012e-04
2,2.298793e-04,9.398738e-07,6.042146e-04,2.512652e-01,7.011834e-01,1.622493e-07,4.670427e-02,6.645349e-06,3.477677e-06,1.782484e-06
3,3.349659e-07,9.999923e-01,2.815372e-07,3.658949e-06,6.982215e-07,5.035546e-07,9.337696e-07,1.129715e-08,1.076237e-06,2.523021e-07
4,8.990123e-01,3.074335e-04,8.232511e-06,3.920937e-02,1.657006e-04,1.175253e-06,6.129311e-02,2.787212e-07,2.170781e-06,1.924311e-07


In [10]:
lst=[]
for i in range(ans.shape[0]):
    lst.append(ans.loc[i].idxmax())

In [11]:
lst

[7,
 7,
 4,
 1,
 0,
 1,
 2,
 0,
 0,
 8,
 8,
 5,
 9,
 0,
 8,
 2,
 5,
 3,
 4,
 0,
 6,
 2,
 6,
 7,
 3,
 8,
 8,
 1,
 3,
 9,
 0,
 1,
 6,
 1,
 7,
 2,
 5,
 3,
 4,
 5,
 7,
 8,
 2,
 5,
 4,
 8,
 4,
 3,
 2,
 8,
 5,
 9,
 1,
 5,
 9,
 5,
 7,
 4,
 2,
 1,
 3,
 4,
 2,
 0,
 8,
 7,
 4,
 8,
 3,
 1,
 7,
 7,
 3,
 7,
 0,
 0,
 6,
 6,
 3,
 4,
 0,
 7,
 1,
 3,
 9,
 6,
 2,
 1,
 5,
 8,
 1,
 3,
 0,
 3,
 5,
 4,
 9,
 8,
 6,
 2,
 9,
 9,
 1,
 8,
 1,
 0,
 0,
 1,
 4,
 6,
 5,
 8,
 8,
 0,
 7,
 4,
 2,
 8,
 2,
 7,
 6,
 5,
 4,
 6,
 0,
 2,
 1,
 5,
 3,
 6,
 8,
 4,
 2,
 6,
 2,
 2,
 2,
 0,
 1,
 4,
 9,
 5,
 2,
 7,
 9,
 7,
 8,
 5,
 7,
 3,
 0,
 3,
 0,
 4,
 8,
 0,
 2,
 4,
 2,
 5,
 6,
 0,
 6,
 0,
 0,
 7,
 1,
 9,
 8,
 8,
 2,
 2,
 7,
 4,
 8,
 0,
 2,
 9,
 8,
 6,
 0,
 8,
 3,
 5,
 4,
 2,
 1,
 0,
 7,
 6,
 5,
 6,
 7,
 2,
 1,
 9,
 0,
 2,
 4,
 7,
 9,
 9,
 4,
 7,
 4,
 2,
 0,
 9,
 5,
 7,
 5,
 7,
 2,
 7,
 3,
 1,
 0,
 9,
 8,
 4,
 0,
 4,
 6,
 8,
 3,
 4,
 5,
 9,
 2,
 2,
 7,
 1,
 1,
 1,
 2,
 3,
 1,
 7,
 7,
 1,
 0,
 2,
 1,
 5,
 1,
 7,
 4,
 9,
 3,
 2,


In [12]:
res=pd.DataFrame(lst)
res.to_pickle("Result.pickle")